In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.commsec import Trades
import datehandler

# Vars
today = datetime.today().date()

In [2]:
trades = Trades()
trades_df = trades.all

p_dates = datehandler.date_list(trades_df.index[-1], today)

tickers = list(sorted(set(trades_df.Ticker.to_list())))
props = ['TradeType','TradeVolume','TradePrice','HoldingVolume','Vwap']

columns = pd.MultiIndex.from_product([tickers,props], names=['Tickers','Props'])
df_p = pd.DataFrame(None, index=pd.DatetimeIndex(p_dates), columns=columns)

# Clean index and columns
df_p.index.name = 'Date'
df_p = df_p.reindex(sorted(df_p.columns),axis=1)  # Sort columns
df_p = df_p.reindex(sorted(df_p.index),axis=0)  # Sort index
df_p

Tickers              A2M                                                 ACW  \
Props      HoldingVolume TradePrice TradeType TradeVolume Vwap HoldingVolume   
Date                                                                           
2015-01-07           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-08           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-09           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-12           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-13           NaN        NaN       NaN         NaN  NaN           NaN   
...                  ...        ...       ...         ...  ...           ...   
2020-01-16           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-17           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-20           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-21           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-22           NaN        NaN       NaN         NaN  NaN           NaN   

Tickers                                           ...           WZR  \
Props      TradePrice TradeType TradeVolume Vwap  ... HoldingVolume   
Date                                              ...                 
2015-01-07        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-08        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-09        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-12        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-13        NaN       NaN         NaN  NaN  ...           NaN   
...               ...       ...         ...  ...  ...           ...   
2020-01-16        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-17        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-20        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-21        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-22        NaN       NaN         NaN  NaN  ...           NaN   

Tickers                                                    ZNO             \
Props      TradePrice TradeType TradeVolume Vwap HoldingVolume TradePrice   
Date                                                                        
2015-01-07        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-08        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-09        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-12        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-13        NaN       NaN         NaN  NaN           NaN        NaN   
...               ...       ...         ...  ...           ...        ...   
2020-01-16        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-17        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-20        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-21        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-22        NaN       NaN         NaN  NaN           NaN        NaN   

Tickers                                
Props      TradeType TradeVolume Vwap  
Date                                   
2015-01-07       NaN         NaN  NaN  
2015-01-08       NaN         NaN  NaN  
2015-01-09       NaN         NaN  NaN  
2015-01-12       NaN         NaN  NaN  
2015-01-13       NaN         NaN  NaN  
...              ...         ...  ...  
2020-01-16       NaN         NaN  NaN  
2020-01-17       NaN         NaN  NaN  
2020-01-20       NaN         NaN  NaN  
2020-01-21       NaN         NaN  NaN  
2020-01-22       NaN         NaN  NaN  

[1316 rows x 175 columns]

In [19]:
dropped_cols = ['Market','Ticker','TradePrice','Brokerage']
for ticker in tickers:
    # Add trade dataframe values to portfolio dataframe
    df_t = trades_df[trades_df.Ticker == ticker].copy()
    df_t.loc[df_t['TradeType'] == 'S', 'Volume'] *= -1  # Don't worry about the warning
    df_t = df_t.drop(columns=dropped_cols)  # Drop columns first
    df_t = df_t.rename(columns={            # Then rename columns - due to conflicting name!
        'Volume':'TradeVolume',
        'EffectivePrice':'TradePrice',
        })
    df_p[ticker] = df_t

In [4]:


# Build portfolio
for ticker in df_p.columns.levels[0]:
    if ticker not in tickers_with_price_data:
        continue

    df_ticker = df_p[ticker].copy()
    
    # HoldingVolume: Cumulative sum for volume
    df_ticker['HoldingVolume'] = df_ticker['TradeVolume'].fillna(0).cumsum()

    # Vwap: BookValue / HoldingVolume
    df_ticker.loc[df_ticker['TradeType'] == 'B','BookValue'] = df_ticker['TradeVolume'] * df_ticker['TradePrice']
    df_ticker.loc[df_ticker['TradeType'] == 'B','BuyVolume'] = df_ticker['TradeVolume']
    df_ticker['BuyVolume'] = df_ticker['BuyVolume'].fillna(0).cumsum()  # Temp calc column
    df_ticker['BookValue'] = df_ticker['BookValue'].fillna(0).cumsum()  # Temp calc column
    df_ticker['Vwap'] = df_ticker['BookValue'] / df_ticker['BuyVolume']

    df_ticker = df_ticker.drop(columns=['BuyVolume','BookValue'])  # Remove temp calc columns
    
    if ticker == 'NAN':
        print(df_ticker)
        df_ticker.to_csv('test.csv')
    

KeyError: 'Ticker'

In [45]:
df_p['NAN']['TradeVolume'].fillna(0).cumsum()

Date
2015-01-07    1000.0
2015-01-08    1000.0
2015-01-09    1000.0
2015-01-12    1000.0
2015-01-13    1000.0
               ...  
2020-01-15    1027.0
2020-01-16    1027.0
2020-01-17    1027.0
2020-01-20    1027.0
2020-01-21    1027.0
Name: TradeVolume, Length: 1315, dtype: float64

In [3]:
import yahoolookup



,Ticker,Market,TradeType,Volume,TradePrice,EffectivePrice,Brokerage,YahooTicker
Date,,,,,,,,
2020-01-16,IBXO,AX,B,100099.0,0.010000,0.010100,10.0,IBXO.AX
2020-01-14,DEM,AX,B,4170.0,0.240000,0.242398,10.0,DEM.AX
2020-01-14,IFN,AX,S,1767.0,0.680000,0.668710,20.0,IFN.AX
2020-01-10,AVH,AX,S,1551.0,0.645000,0.638556,10.0,AVH.AX
2020-01-10,VRS,AX,S,4553.0,0.064000,0.061803,10.0,VRS.AX
...,...,...,...,...,...,...,...,...
2017-08-03,NAN,AX,B,2000.0,2.310700,2.320675,20.0,NAN.AX
2016-07-06,QBE,AX,B,500.0,10.280000,10.319900,20.0,QBE.AX
2016-04-28,RMD,AX,B,300.0,7.500000,7.566500,20.0,RMD.AX
